#### Análisis Multivariado

En primer lugar es necesario cuantificar las columnas con datos de tipo ``object`` para poder ser tratados matemáticamente.

In [ ]:
hotel.info()

In [ ]:
hotel_analisis_multivariado = hotel.copy()

hotel_analisis_multivariado['hotel'] = hotel_analisis_multivariado['hotel'].map(
    {'Resort Hotel': 0, 'City Hotel': 1})

hotel_analisis_multivariado['arrival_date_month'] = hotel_analisis_multivariado['arrival_date_month'].map(
    {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12})

hotel_analisis_multivariado['meal'] = hotel_analisis_multivariado['meal'].map(
    {'Undefined': 0, 'FB': 1, 'HB': 2, 'SC': 3, 'BB': 4})

hotel_analisis_multivariado['market_segment'] = hotel_analisis_multivariado['market_segment'].map(
    {'Undefined': 0, 'Direct': 1, 'Corporate': 2, 'Online TA': 3, 'Offline TA/TO': 4, 'Complementary': 5, 'Groups': 6, 'Aviation': 7})

hotel_analisis_multivariado['distribution_channel'] = hotel_analisis_multivariado['distribution_channel'].map(
    {'Undefined': 0, 'Direct': 1, 'Corporate': 2, 'TA/TO': 3, 'GDS': 4})

hotel_analisis_multivariado['reserved_room_type'] = hotel_analisis_multivariado['reserved_room_type'].map(
    {'A': 1, 'D': 2, 'E': 3, 'G': 4, 'F': 5, 'H': 6, 'L': 7, 'B': 8, 'C': 9})

hotel_analisis_multivariado['assigned_room_type'] = hotel_analisis_multivariado['assigned_room_type'].map(
    {'A': 1, 'D': 2, 'E': 3, 'G': 4, 'F': 5, 'H': 6, 'L': 7, 'K': 8, 'B': 9, 'C': 10, 'I': 11})

hotel_analisis_multivariado['deposit_type'] = hotel_analisis_multivariado['deposit_type'].map(
    {'No Deposit': 0, 'Non Refund': 1, 'Refundable': 2})

hotel_analisis_multivariado['customer_type'] = hotel_analisis_multivariado['customer_type'].map(
    {'Transient': 0, 'Contract': 1, 'Transient-Party': 2, 'Group': 3})

hotel_analisis_multivariado.drop(
    columns=['agent', 'company', 'id', 'country', 'arrival_date'], inplace=True)


In [ ]:
hotel_analisis_multivariado.info()

##### Multivariado - Isolation forest

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
hotel_isolation_forest = hotel_analisis_multivariado.copy()
X = hotel_isolation_forest.copy().drop(columns=['is_canceled'])
y = hotel_isolation_forest['is_canceled']
# select the features

# X = data[['adr', 'children', 'babies', 'adults', 'previous_cancellations']]
# y = data['is_canceled']
hotel_isolation_forest.head()

In [ ]:
iforest = IsolationForest(n_estimators=100, max_samples='auto',
                          contamination=0.05, max_features=4,
                          bootstrap=False, n_jobs=-1, random_state=1)


In [ ]:
pred = iforest.fit_predict(X)

In [ ]:
#Almaceno scores y etiquetas
hotel_isolation_forest['scores'] = iforest.decision_function(X)
hotel_isolation_forest['outlier_label'] = pred

#Observaciones anómalas
hotel_isolation_forest[hotel_isolation_forest.outlier_label == -1]

In [ ]:
# count the number of outliers
hotel_isolation_forest.outlier_label.value_counts()

##### Multivariado - Local Outlier Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
hotel_lof = hotel_analisis_multivariado.copy()

hotel_lof.head()

In [ ]:
X = hotel_lof.copy().drop(columns=['is_canceled']).values

clf = LocalOutlierFactor(n_neighbors=20)

pred = clf.fit_predict(X)

hotel_lof['scores'] = clf.negative_outlier_factor_

In [ ]:
hotel_lof.scores.max(), hotel_lof.scores.min()

In [ ]:
# count the number of registers with a score less than -1.5
hotel_lof[hotel_lof.scores < -1.5].shape[0]